In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import numpy as np
import os
import random
import shutil
import matplotlib.pyplot as plt
import cv2

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import image_dataset_from_directory
from keras.preprocessing.image import ImageDataGenerator


In [ ]:
train_directory = '/content/drive/MyDrive/Datasets/rim/augmented train'
test_directory = '/content/drive/MyDrive/Datasets/rim/test'
val_directory = '/content/drive/MyDrive/Datasets/rim/val'


In [ ]:
categories = os.listdir(train_directory)
print(str(len(categories)),'CATEGORIES are ', categories)

category_count = len(categories)


4 CATEGORIES are  ['kidney cyst', 'kidney tumor', 'kidney stone', 'kidney normal']


In [ ]:
def preprocess_image(image):
    if image.shape[-1] == 3:
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray_image = image
    return gray_image

augmented_gen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=preprocess_image
)

general_datagen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=preprocess_image
)

train_generator = general_datagen.flow_from_directory(
    train_directory,
    target_size=(224, 224),
    batch_size=32,
    color_mode='grayscale'  # Define o modo de cor como escala de cinza
)

valid_generator = general_datagen.flow_from_directory(
    val_directory,
    target_size=(224, 224),
    batch_size=32,
    color_mode='grayscale'  # Define o modo de cor como escala de cinza
)

test_generator = general_datagen.flow_from_directory(
    test_directory,
    target_size=(224, 224),
    batch_size=32,
    color_mode='grayscale',  # Define o modo de cor como escala de cinza
    shuffle=False
)


Found 8005 images belonging to 4 classes.
Found 1865 images belonging to 4 classes.
Found 1871 images belonging to 4 classes.


In [ ]:
diretorio = "/content/drive/MyDrive/Datasets/rim/augmented train/kidney tumor"

# Lista de todas as imagens no diretório
imagens = os.listdir(diretorio)

# Calcula o número de imagens a serem excluídas (metade das imagens)
numero_de_imagens_para_excluir = len(imagens)
print(numero_de_imagens_para_excluir)


In [ ]:
train_groups = len(train_generator)
valid_groups = len(valid_generator)
test_groups = len(test_generator)
print(f"Train groups: {train_groups}")
print(f"Validation groups: {valid_groups}")
print(f"Test groups: {test_groups}")

Train groups: 251
Validation groups: 59
Test groups: 59


In [ ]:
def conv_layer(inputs, filters, kernel_size=3, padding="valid"):
    x = layers.Conv2D(filters = filters, kernel_size = kernel_size, padding = padding, use_bias = False)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    return x
# pooling layer i added dropout cause it help my model to reduce the overfitting
def pooling_layer(inputs, pool_size = 2, dropout_rate=0.5):
    x = layers.MaxPooling2D(pool_size = pool_size)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout_rate)(x)

    return x

# this dense layer i will not only use it for my base model i will use it in the pretrained model too
def dense_layer(inputs, out, dropout_rate = 0.5):
    x = layers.Dense(out)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.Dropout(dropout_rate)(x)

    return x


In [ ]:
keras.backend.clear_session()

inputs = keras.Input(shape = (224, 224, 1))

x = conv_layer(inputs, 64, padding = "same")  # 224x224
x = conv_layer(x, 64)                         # 222x222
x = pooling_layer(x)                          # 111x111

x = conv_layer(x, 64, padding = "same")       # 111x111
x = conv_layer(x, 64)                         # 109x109
x = pooling_layer(x)                          # 54x54

x = conv_layer(x, 64, padding = "same")       # 54x54
x = conv_layer(x, 64)                         # 52X52
x = pooling_layer(x)                          # 26x26

x = conv_layer(x, 64, padding = "same")       # 26x26

x = layers.Flatten()(x)                       # 26*26*64

x = dense_layer(x, 96)

outputs = layers.Dense(category_count, activation = "softmax")(x)

base_model = keras.Model(inputs, outputs)

base_model.summary()

In [ ]:
base_model.compile(optimizer =keras.optimizers.Adam(learning_rate=0.001),
               loss = 'categorical_crossentropy',
               metrics = ['accuracy'])
#fit model
history = base_model.fit(
    train_generator,
    steps_per_epoch = train_groups,
    epochs = 20, # adding more epochs will increase the acc like 1% or 2%
    validation_data = valid_generator,
    validation_steps = valid_groups,
    verbose = 1,
    callbacks=[keras.callbacks.EarlyStopping(monitor='val_accuracy', patience = 10, restore_best_weights = True),
               keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss', factor = 0.7, patience = 2, verbose = 1),
    keras.callbacks.ModelCheckpoint(
            filepath = "/content/drive/MyDrive/Datasets/rim/intial_model.h5",
            save_best_only = True,
            monitor = "val_loss")
    ])

Epoch 1/20
251/251 [==============================] - ETA: 0s - loss: 1.0808 - accuracy: 0.5315 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


251/251 [==============================] - 6649s 26s/step - loss: 1.0808 - accuracy: 0.5315 - val_loss: 3.5394 - val_accuracy: 0.2987 - lr: 0.0010
Epoch 2/20
251/251 [==============================] - 108s 432ms/step - loss: 0.7633 - accuracy: 0.6756 - val_loss: 2.3595 - val_accuracy: 0.2633 - lr: 0.0010
Epoch 3/20
251/251 [==============================] - 105s 420ms/step - loss: 0.6213 - accuracy: 0.7484 - val_loss: 0.5873 - val_accuracy: 0.7673 - lr: 0.0010
Epoch 4/20
251/251 [==============================] - 104s 415ms/step - loss: 0.5201 - accuracy: 0.7916 - val_loss: 1.4173 - val_accuracy: 0.5228 - lr: 0.0010
Epoch 5/20
251/251 [==============================] - 109s 436ms/step - loss: 0.4437 - accuracy: 0.8274 - val_loss: 0.3485 - val_accuracy: 0.8708 - lr: 0.0010
Epoch 6/20
251/251 [==============================] - 104s 415ms/step - loss: 0.4091 - accuracy: 0.8442 - val_loss: 0.3898 - val_accuracy: 0.8488 - lr: 0.0010
Epoch 7/20
251/251 [==============================] - ETA:

In [ ]:
accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]

loss = history.history["loss"]
val_loss = history.history["val_loss"]


print(accuracy[19])
#epochs = range(1, len(accuracy) + 1)

#plt.plot(epochs, accuracy, "bo", label = "Trianing accuracy")
#plt.plot(epochs, val_accuracy, "b-", label = "Validation accuracy")
#plt.title("Accuracy on training and validation data")
#plt.legend()
#plt.figure()

#plt.plot(epochs, loss, "bo", label = "Trianing loss")
#plt.plot(epochs, val_loss, "b-", label = "Validation loss")
#plt.title("loss on training and validation data")
#plt.title("loss on training and validation data")
#plt.legend()
#plt.show()

In [ ]:


model = keras.models.load_model("/content/drive/MyDrive/Datasets/rim/intial_model.h5")


test_results = model.evaluate(test_generator)


loss, accuracy = test_results

print(f'Perda (Loss): {loss}')
print(f'Acurácia: {accuracy}')

59/59 [==============================] - 447s 8s/step - loss: 0.1137 - accuracy: 0.9519
Perda (Loss): 0.11367297917604446
Acurácia: 0.951897382736206


In [6]:
predictions = model.predict(test_generator)


predicted_classes = np.argmax(predictions, axis=1)


true_classes = test_generator.classes


from sklearn.metrics import accuracy_score
accuracy = accuracy_score(true_classes, predicted_classes)
print(f'Precisão: {accuracy}')

59/59 [==============================] - 306s 5s/step
Precisão: 0.9518973810796365


In [7]:
from sklearn.metrics import f1_score
import numpy as np


predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)


true_classes = test_generator.classes


f1 = f1_score(true_classes, predicted_classes, average='weighted')

print(f'F1-Score: {f1}')

59/59 [==============================] - 283s 5s/step
F1-Score: 0.9527755078463199


In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np


predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)


true_classes = test_generator.classes


confusion = confusion_matrix(true_classes, predicted_classes)

print('Matriz de Confusão:')
print(confusion)

17/59 [=======>......................] - ETA: 3:21